# PlanetaRY spanGLES

This is the initialization file of the `Pryngles` package.  

> **Note for `JupDev`**: This is the only module that should be converted as it is without using any decorator `#@``

## Packages common to all modules

In [1]:
#@external:__init__
import unittest
import warnings
import dill
import inspect
import sigfig
from copy import deepcopy
import sys
from collections import OrderedDict as odict
warnings.filterwarnings('ignore')

#JupDev: Jupyter compatibility
from IPython.display import HTML, Image, display
import IPython.core.autocall as autocall
from IPython import get_ipython
#@end:external

## Jupyter compatibilty

In [2]:
#@standalone
"""
The purpose of the get_ipython class is to provide some response in the python 
script resulting from the conversion of this notebook.

If you want to add another IPyhton function resulting from a magic command to the class, 
please verify in the resulting python script the corresponding IPython command.

For instance, the magic "%matplotlib nbagg" is converted into:

    get_ipython().magic('matplotlib nbagg',globals())

So, the method "magic" should be add to the get_ipython() class.        
"""
try:
    cfg=get_ipython().config
except AttributeError:
    def Image(url="",filename="",f=""):
        pass
    class get_ipython(object):
        def run_line_magic(self,*args):
            pass
        def run_cell_magic(self,x,y,z):
            pass
        def magic(self,command,scope=globals()):
            pass

#Magics can only be located starting from here
%load_ext autoreload
%autoreload 2
#@end:standalone

## Verbosity

In [3]:
#@standalone
#Verbosity levels: see help(Verbose)
VERB_NONE=0
VERB_SIMPLE=1
VERB_SYSTEM=2
VERB_VERIFY=3
VERB_DEEP=4
VERB_ALL=100

class Verbose(object):
    """Verbose print in the package
    
    Attributes:
        VERBOSITY: int, default = 0:
            Level of verbosity.
            
            Verbosity levels:
                SIMPLE: Simple messages.
                SYSTEM: System operations.
                VERIFY: Message to verify operations
                DEEP: Deep debugging messages
                ALL: All debugging messages
                
    Methods:
        print(level,msg):
            Print a message if level<=VERBOSITY.
    
    Example:
    
        Verbose.print(1,"Hello world") #No print
        
        Verbose.print(0,"Hello world") #Print

        Verbose.VERBOSITY=1
        Verbose.print(1,"Hello world") #Print
        
        Verbose.VERBOSITY=2
        Verbose.print(1,"Hello world") #Print
        
        Verbose.VERBOSITY=2
        Verbose.print(4,"Hello world") #No print
    """
    VERBOSITY=VERB_ALL
    def print(level,*args):
        if level<=Verbose.VERBOSITY:
            print("  "*level+f"VERB{level}::{inspect.stack()[1][3]}::",*args)
            
#Alias
verbose=Verbose.print
#@end:standalone

## PrynglesCommon

Many of the classes in Pryngles inherite methods of this common class

In [4]:
#@standalone
class PrynglesCommon(object):
    """Base class of the package.
    
    All major classes are children of PrynglesCommon class.
    """
    def __init__(self):
        pass
    
    def save_to(self,filename):
        """Save object to a binary file
        
        Parameters:
            filename: string:
                Name of the file where the object will be stored.
        
        Notes:
            Based on https://betterprogramming.pub/load-fast-load-big-with-compressed-pickles-5f311584507e.
        """
        verbose(VERB_SYSTEM,f"Saving object to {filename}")
        pikd = open(filename,"wb")
        dill.dump(self, pikd)
        pikd.close()
            
    def load_from(self,filename):
        """Read object from a binary file.
        
        Parameters:
            filename: string:
                Name of the file where the object is stored.        
        """
        verbose(VERB_SYSTEM,f"Loading object from {filename}")
        pikd = open(filename,"rb")
        data = dill.load(pikd)
        pikd.close()
        verbose(VERB_VERIFY,f"Transferring data to new object")
        self.__dict__=data.__dict__
        return data
    
    def __str__(self):
        """Show content of an object
        
        This method determines the default behavior of the command:
        
            print(object)
        """
        #Remove private attributes
        return str({k:v for k,v in self.__dict__.items() if k[0]!='_'})
#@end:standalone

## Pryngles modules

In [5]:
#@standalone
from pryngles.version import *

#Constants
from pryngles.consts import *

#Utility modules
from pryngles.misc import *
from pryngles.extensions import *
from pryngles.science import *
from pryngles.plot import *
from pryngles.orbit import *
from pryngles.scatterer import *

#Legacy module
from pryngles.legacy import *

#Core modules
from pryngles.sampler import *
from pryngles.spangler import *
from pryngles.body import *
from pryngles.system import *

#Reset verbosity
Verbose.VERBOSITY=VERB_NONE
#@end:standalone

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Aliases

In [6]:
#@standalone
#This aliases does not work in modules
print_df=Misc.print_df
sci=Science
#@end:standalone

## Tests

In [7]:
#@test:__init__
def test_common(self):
    import numpy as np
    import pandas as pd
    import os

    Verbose.VERBOSITY=VERB_ALL

    p=PrynglesCommon()
    p.casa=dict(perro=0,gato=3)
    p.data=pd.DataFrame(np.random.rand(4000,100))
    p.save_to("/tmp/save.pck")
    print("File size:",os.path.getsize("/tmp/save.pck")/1e6," Mb")
    g=PrynglesCommon()
    g.load_from("/tmp/save.pck")
    print(g.casa,np.array(g.data).shape)

    Verbose.VERBOSITY=VERB_NONE

class Test(unittest.TestCase):pass    
Test.test_common=test_common
unittest.main(argv=['first-arg-is-ignored'],exit=False)
#@end:test

.

    VERB2::save_to:: Saving object to /tmp/save.pck
File size: 3.200736  Mb
    VERB2::load_from:: Loading object from /tmp/save.pck
      VERB3::load_from:: Transferring data to new object
{'perro': 0, 'gato': 3} (4000, 100)



----------------------------------------------------------------------
Ran 1 test in 0.132s

OK


### The end

In [8]:
#@end:module

## Playing ground